In [ ]:
data = pd.read_excel("dataset.xlsx")                              # Load data (feature-target)
x = data.drop(columns=["target"])
y = data["target"]

x = RobustScaler().fit_transform(x)                               # Scaling, split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)



param_grid = {                                                    # XGB
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}
xgb = XGBRegressor()
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(x_train, y_train)
best_xgb = grid_search.best_estimator_
best_xgb.fit(x_train, y_train)
y_pred_test_xgb  = best_xgb.predict(x_test)
print('R2       :', r2_score(y_test, y_pred_test_xgb))
print('RMSE     :', np.sqrt(mean_squared_error(y_test, y_pred_test_xgb)))



backend        = Aer.get_backend('qasm_simulator')                       # QSVR
feature_map    = ZFeatureMap(feature_dimension=x_train.shape[1], reps=1)
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)
qsvr = QSVR(quantum_kernel=quantum_kernel)
qsvr.fit(x_train, y_train)

y_pred_test_qsvr = qsvr.predict(x_test)
print('R2       :', r2_score(y_test, y_pred_test_qsvr))
print('RMSE     :', np.sqrt(mean_squared_error(y_test, y_pred_test_qsvr)))